In [1]:
import numpy as np
import lightgbm as lgb
from wideboost.wrappers import wlgb

import tensorflow as tf
import tensorflow_datasets as tfds

ds_train, ds_info = tfds.load(
    'titanic',
    split=['train'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train[0].batch(60000):
    a = i

In [2]:
## Keep only numeric columns. Randomly split the dataset into training and test

X = []
for s in a[0].keys():
    if a[0][s].dtype != tf.string:
        X.append(a[0][s].numpy().reshape([-1,1]))

X = np.concatenate(X,axis=1)

idx = np.arange(X.shape[0])
np.random.seed(1)
train_idx = np.random.choice(idx, round(0.7*X.shape[0]), replace=False)
test_idx = np.setdiff1d(idx,train_idx)

xtrain = X[train_idx,:]
ytrain = a[1].numpy()[train_idx]

xtest = X[test_idx,:]
ytest = a[1].numpy()[test_idx]

train_data = lgb.Dataset(xtrain, label=ytrain)
test_data = lgb.Dataset(xtest, label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    #watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    param['num_leaves'] = round(param['num_leaves']+1)
    param['min_data_in_leaf'] = round(param['min_data_in_leaf'])
    wbst = lgb.train(param,
                      train_data,
                      num_boost_round=100,
                      valid_sets=test_data,
                      evals_result=ed1_results)
    output = min(ed1_results['valid_0']['binary_error'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'objective': hp.choice('objective',['binary']),
    'metric':hp.choice('metric',['binary_error']),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'num_leaves' : hp.qloguniform('num_leaves', 0, 7, 1),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
    'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 0, 6, 1),
    'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', -16, 5),
    'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
    'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)])
}


best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'bagging_fraction': 0.5072835875210108, 'feature_fraction': 0.7059679536259044, 'lambda_l1': 0, 'lambda_l2': 0, 'learning_rate': 0.006188246691481022, 'metric': 'binary_error', 'min_data_in_leaf': 169.0, 'min_sum_hessian_in_leaf': 0.037204142106990766, 'num_leaves': 304.0, 'objective': 'binary'}
[1]	valid_0's binary_error: 0.384224                   
[2]	valid_0's binary_error: 0.384224                   
[3]	valid_0's binary_error: 0.384224                   
[4]	valid_0's binary_error: 0.384224                   
[5]	valid_0's binary_error: 0.384224                   
[6]	valid_0's binary_error: 0.384224                   
[7]	valid_0's binary_error: 0.384224                   
[8]	valid_0's binary_error: 0.384224                   
[9]	valid_0's binary_error: 0.384224                   
[10]	valid_0's binary_error: 0.384224                  
[11]	valid_0's binary_error: 0.384224                  
[12]	valid_0's binary_error: 0.384224                  
[13]	valid_0's binary_error: 0

[24]	valid_0's binary_error: 0.384224                                             
[25]	valid_0's binary_error: 0.384224                                             
[26]	valid_0's binary_error: 0.384224                                             
[27]	valid_0's binary_error: 0.384224                                             
[28]	valid_0's binary_error: 0.384224                                             
[29]	valid_0's binary_error: 0.384224                                             
[30]	valid_0's binary_error: 0.384224                                             
[31]	valid_0's binary_error: 0.384224                                             
[32]	valid_0's binary_error: 0.384224                                             
[33]	valid_0's binary_error: 0.384224                                             
[34]	valid_0's binary_error: 0.384224                                             
[35]	valid_0's binary_error: 0.384224                                             
[36]

[19]	valid_0's binary_error: 0.195929                                             
[20]	valid_0's binary_error: 0.198473                                             
[21]	valid_0's binary_error: 0.195929                                             
[22]	valid_0's binary_error: 0.188295                                             
[23]	valid_0's binary_error: 0.19084                                              
[24]	valid_0's binary_error: 0.195929                                             
[25]	valid_0's binary_error: 0.198473                                             
[26]	valid_0's binary_error: 0.193384                                             
[27]	valid_0's binary_error: 0.193384                                             
[28]	valid_0's binary_error: 0.193384                                             
[29]	valid_0's binary_error: 0.195929                                             
[30]	valid_0's binary_error: 0.198473                                             
[31]

[12]	valid_0's binary_error: 0.201018                                             
[13]	valid_0's binary_error: 0.201018                                             
[14]	valid_0's binary_error: 0.203562                                             
[15]	valid_0's binary_error: 0.203562                                             
[16]	valid_0's binary_error: 0.206107                                             
[17]	valid_0's binary_error: 0.201018                                             
[18]	valid_0's binary_error: 0.198473                                             
[19]	valid_0's binary_error: 0.198473                                             
[20]	valid_0's binary_error: 0.198473                                             
[21]	valid_0's binary_error: 0.195929                                             
[22]	valid_0's binary_error: 0.195929                                             
[23]	valid_0's binary_error: 0.193384                                             
[24]

[5]	valid_0's binary_error: 0.198473                                              
[6]	valid_0's binary_error: 0.19084                                               
[7]	valid_0's binary_error: 0.19084                                               
[8]	valid_0's binary_error: 0.185751                                              
[9]	valid_0's binary_error: 0.180662                                              
[10]	valid_0's binary_error: 0.19084                                              
[11]	valid_0's binary_error: 0.183206                                             
[12]	valid_0's binary_error: 0.188295                                             
[13]	valid_0's binary_error: 0.183206                                             
[14]	valid_0's binary_error: 0.188295                                             
[15]	valid_0's binary_error: 0.185751                                             
[16]	valid_0's binary_error: 0.180662                                             
[17]

...

[3]	valid_0's binary_error: 0.195929                                              
[4]	valid_0's binary_error: 0.195929                                              
[5]	valid_0's binary_error: 0.195929                                              
[6]	valid_0's binary_error: 0.195929                                              
[7]	valid_0's binary_error: 0.195929                                              
[8]	valid_0's binary_error: 0.195929                                              
[9]	valid_0's binary_error: 0.195929                                              
[10]	valid_0's binary_error: 0.208651                                             
[11]	valid_0's binary_error: 0.208651                                             
[12]	valid_0's binary_error: 0.195929                                             
[13]	valid_0's binary_error: 0.208651                                             
[14]	valid_0's binary_error: 0.198473                                             
[15]

{'bagging_fraction': 0.57667955267029, 'feature_fraction': 0.5693014888804749, 'lambda_l1': 0.0003273343430200239, 'lambda_l2': 0, 'learning_rate': 0.1298932223841838, 'metric': 'binary_error', 'min_data_in_leaf': 39.0, 'min_sum_hessian_in_leaf': 2.3945671129455912e-05, 'num_leaves': 287.0, 'objective': 'binary'}
[1]	valid_0's binary_error: 0.384224                                              
[2]	valid_0's binary_error: 0.320611                                              
[3]	valid_0's binary_error: 0.244275                                              
[4]	valid_0's binary_error: 0.231552                                              
[5]	valid_0's binary_error: 0.226463                                              
[6]	valid_0's binary_error: 0.221374                                              
[7]	valid_0's binary_error: 0.221374                                              
[8]	valid_0's binary_error: 0.216285                                              
[9]	valid_0's binary_

[95]	valid_0's binary_error: 0.180662                                             
[96]	valid_0's binary_error: 0.180662                                             
[97]	valid_0's binary_error: 0.175573                                             
[98]	valid_0's binary_error: 0.178117                                             
[99]	valid_0's binary_error: 0.183206                                             
[100]	valid_0's binary_error: 0.180662                                            
{'bagging_fraction': 0.6567037717527583, 'feature_fraction': 0.6393403120446908, 'lambda_l1': 0.003038663566813418, 'lambda_l2': 0, 'learning_rate': 0.09361438378060483, 'metric': 'binary_error', 'min_data_in_leaf': 59.0, 'min_sum_hessian_in_leaf': 1.1681602213998047e-05, 'num_leaves': 704.0, 'objective': 'binary'}
[1]	valid_0's binary_error: 0.384224                                              
[2]	valid_0's binary_error: 0.384224                                              
[3]	valid_0's binar

[89]	valid_0's binary_error: 0.178117                                             
[90]	valid_0's binary_error: 0.178117                                             
[91]	valid_0's binary_error: 0.178117                                             
[92]	valid_0's binary_error: 0.178117                                             
[93]	valid_0's binary_error: 0.178117                                             
[94]	valid_0's binary_error: 0.178117                                             
[95]	valid_0's binary_error: 0.178117                                             
[96]	valid_0's binary_error: 0.178117                                             
[97]	valid_0's binary_error: 0.175573                                             
[98]	valid_0's binary_error: 0.175573                                             
[99]	valid_0's binary_error: 0.175573                                             
[100]	valid_0's binary_error: 0.178117                                            
{'ba

[83]	valid_0's binary_error: 0.203562                                             
[84]	valid_0's binary_error: 0.201018                                             
[85]	valid_0's binary_error: 0.201018                                             
[86]	valid_0's binary_error: 0.198473                                             
[87]	valid_0's binary_error: 0.198473                                             
[88]	valid_0's binary_error: 0.195929                                             
[89]	valid_0's binary_error: 0.195929                                             
[90]	valid_0's binary_error: 0.195929                                             
[91]	valid_0's binary_error: 0.193384                                             
[92]	valid_0's binary_error: 0.188295                                             
[93]	valid_0's binary_error: 0.188295                                             
[94]	valid_0's binary_error: 0.185751                                             
[95]

In [4]:
print(best_val)
print(space_eval(spc, best))

0.1628498727735369
{'bagging_fraction': 0.6222218859368742, 'feature_fraction': 0.8084370118138293, 'lambda_l1': 0.01829698717818334, 'lambda_l2': 0, 'learning_rate': 0.4325756117747774, 'metric': 'binary_error', 'min_data_in_leaf': 37.0, 'min_sum_hessian_in_leaf': 1.2285771777805918e-06, 'num_leaves': 908.0, 'objective': 'binary'}
